In [38]:
import pandas as pd
import numpy as np
import emoji
import datetime


In [40]:
dfg = pd.read_csv("/home/bosser/Documentos/PROYECTOFINAL/CSV/CSV_finales/ofertas_final.csv")

In [42]:
dfg

,id,empresa,titulo,ciudad,salario_desde,salario_hasta,variable,vacaciones,teletrabajo,jornada_tipo,...,nivel_profesional,formacion_minima,personas_a_cargo,experiencia,vacantes,cvs_inscritos,oferta_activada,fecha_publicacion,timestamp,url
0,id_tec_1,NPR Spain,UrgenteArquitecto Soluciones Observabilidad,100% remoto,NaN,NaN,NaN,NaN,Presencial,Jornada Completa,...,Empleado,Grado Medio,NaN,Más de 5 años,NaN,20.0,1,2025-02-21,2025-02-22 17:03:04,https://www.tecnoempleo.com/arquitecto-solucio...
1,id_tec_2,Infinity Neural,UrgenteData Analyst BI - Junior,Barcelona,NaN,NaN,NaN,NaN,Hibrido,Jornada Completa,...,Empleado,Grado EEES (Bolonia),NaN,2 años,NaN,23.0,1,2025-02-21,2025-02-22 17:03:05,https://www.tecnoempleo.com/data-analyst-bi-ju...
2,id_tec_3,Talentea Recruitment,UrgenteIT Project Manager,Lleida,36.0,60.0,NaN,NaN,Presencial,Jornada Completa,...,Mando Intermedio,Ingeniero Técnico,NaN,3-5 años,NaN,4.0,1,2025-02-21,2025-02-22 17:03:06,https://www.tecnoempleo.com/it-project-manager...
3,id_tec_4,Beskar Forge,UrgenteSysAdmin/XOps Junior,Madrid,27.0,30.0,NaN,NaN,Hibrido,Jornada Completa,...,Empleado,FP2/Grado Superior,NaN,2 años,2.0,44.0,1,2025-02-21,2025-02-22 17:03:07,https://www.tecnoempleo.com/sysadmin-xops-juni...
4,id_tec_5,DAC.digital,UrgenteIT System Administrator/Web Developer,100% remoto,32.4,72.0,NaN,NaN,Presencial,Jornada Completa,...,Especialista,Sin estudios,NaN,3-5 años,NaN,116.0,1,2025-02-21,2025-02-22 17:03:07,https://www.tecnoempleo.com/it-system-administ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4825,id_man_1222,product,Frontend Developer Junior,NaN,24.0,26.0,0.0,22.0,Remoto,Jornada Completa,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,2025-02-14 12:00:22,https://www.getmanfred.com/ofertas-empleo/27/p...
4826,id_man_1223,foundernest,Senior Backend Developer,MADRID,50.0,60.0,0.0,22.0,Remoto,Jornada Completa,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,2025-02-14 12:00:23,https://www.getmanfred.com/ofertas-empleo/23/f...
4827,id_man_1224,mindcurv,Senior Java Developer,MADRID,45.0,55.0,0.0,30.0,Remoto,Jornada Completa,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,2025-02-14 12:00:25,https://www.getmanfred.com/ofertas-empleo/21/m...
4828,id_man_1225,mindcurv,Solution Architect,NaN,80.0,80.0,0.0,30.0,Remoto,Jornada Completa,...,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,2025-02-14 12:00:27,https://www.getmanfred.com/ofertas-empleo/18/m...


In [44]:
dfg = pd.read_csv("/home/bosser/Documentos/PROYECTOFINAL/CSV/CSV_finales/ofertas_final.csv")
df_pres = dfg[["id", "ciudad"]]  # Extraemos las columnas necesarias del DataFrame original

def split_presencial(row):
    # Verificar si "Presencial" no es NaN y contiene "OR" o "O"
    if pd.notna(row["ciudad"]) and (" OR " in row["ciudad"] or " O " in row["ciudad"]):
        ciudades = [ciudad.strip() for ciudad in row["ciudad"].replace(" O ", " OR ").split(" OR ")]
        return pd.DataFrame({
            "id": [row["id"]] * len(ciudades),
            "ciudades": ciudades
        })
    else:
        return pd.DataFrame({"id": [row["id"]], "ciudades": [row["ciudad"]]})

# Aplicar la función y concatenamos el resultado
result_df = pd.concat(df_pres.apply(split_presencial, axis=1).tolist(), ignore_index=True)
result_df.to_csv("resultado_presencial.csv", index=False)

In [46]:
result_df

,id,ciudades
0,id_tec_1,100% remoto
1,id_tec_2,Barcelona
2,id_tec_3,Lleida
3,id_tec_4,Madrid
4,id_tec_5,100% remoto
...,...,...
4862,id_man_1222,NaN
4863,id_man_1223,MADRID
4864,id_man_1224,MADRID
4865,id_man_1225,NaN


In [60]:
import pandas as pd
from geopy.geocoders import Nominatim
from time import sleep

ciudades = pd.read_csv("/home/bosser/Documentos/PROYECTOFINAL/CSV/CSV_finales/resultado_presencial.csv")
lista_ciudades = [i for i in ciudades["ciudades"]]
ciudades = list(set(lista_ciudades))

In [78]:
def obtener_coordenadas(ciudades, nombre_archivo="ciudades_coordenadas.csv"):
    # Inicializamos el geolocalizador
    geolocalizador = Nominatim(user_agent="geoapi")
    
    # Lista para almacenar los datos
    datos = []

    for ciudad in ciudades:
        try:
            ubicacion = geolocalizador.geocode(ciudad, timeout=10, addressdetails=True)
            if ubicacion:
                direccion = ubicacion.raw.get("address", {})
                datos.append({
                    "Ciudad": ciudad.title(),
                    "País": direccion.get("country", "Desconocido"),
                    "Comunidad Autónoma": direccion.get("state", "Desconocido"),
                    "Latitud": ubicacion.latitude,
                    "Longitud": ubicacion.longitude
                })
            else:
                print(f"No se encontraron coordenadas para: {ciudad}")
            sleep(1)  # Esperar para evitar bloqueos de la API
        except Exception as e:
            print(f"Error con {ciudad}: {e}")

    # Convertimos a DataFrame e ignoramos filas con 
    df = pd.DataFrame(datos).dropna()

    # Guardamos en CSV
    df.to_csv(nombre_archivo, index=False)
    print(f"Archivo guardado: {nombre_archivo}")
    
    return df
    

In [80]:
obtener_coordenadas(ciudades)

Error con nan: 'float' object has no attribute 'title'
No se encontraron coordenadas para: Rep. De Macedonia
Archivo guardado: ciudades_coordenadas.csv


,Ciudad,País,Comunidad Autónoma,Latitud,Longitud
0,Madrid,España,Comunidad de Madrid,40.416705,-3.703582
1,Bizkaia,España,Euskadi,43.238482,-2.851632
2,"Milán, Italia",Italia,Lombardia,45.464194,9.189635
3,Oporto,Portugal,Desconocido,41.149451,-8.610788
4,Polonia,Polska,Desconocido,52.215933,19.134422
...,...,...,...,...,...
86,Murcia,España,Región de Murcia,37.992379,-1.130543
87,Alcoy,España,Comunitat Valenciana,38.698235,-0.474762
88,Zaragoza,España,Aragón,41.652134,-0.880943
89,Almería,España,Andalucía,36.841420,-2.462814


In [86]:
vista01 = pd.read_csv("/home/bosser/Documentos/PROYECTOFINAL/CSV/CSV_finales/ciudades_coordenadas.csv")

In [88]:
vista01["Comunidad Autónoma"].unique()


array(['Comunidad de Madrid', 'Euskadi', 'Lombardia', 'Desconocido',
       'Plazas de Soberanía', 'Castilla-La Mancha', 'Castilla y León',
       'Galicia', 'Aragón', 'Catalunya', 'Cantabria', 'Région de Dakar',
       'Andalucía', 'Asturias / Asturies', 'La Rioja', 'Illes Balears',
       'Comunitat Valenciana', 'Región de Murcia', 'Jalisco', 'Melilla',
       'England', 'Extremadura', 'Noord-Holland', 'Navarra - Nafarroa',
       'Κύπρος', 'Ciudad de México', 'La Guajira', 'Bayern'], dtype=object)